# Modelling with lasagne

## Import and options

In [ ]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [ ]:
import h5py
import dask
# from dask import array as da
from dask import dataframe as dd
# from dask import delayed
# from dask.multiprocessing import get
import pandas as pd
import pathlib2 as pl
import mmh3  # The hash function used to hash sites. See the preprocessor script.

In [ ]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 6)
pd.set_option('display.width', 1000)

# dask.set_options(get=get);  # Due to a bug we can't read files in different processes so set this option after reading.

## Loading data into dask dataframes

Our preprocessor supports output into `numpy` `arrays` and `pandas` `DataFrames` and `scikit-learn` supports the latter.

In [ ]:
CHUNK_SIZE = int(1e5)
DF_DIR = pl.Path('/Volumes/CompanionEx/Data/dfs_pandas/PP_TS_2016-05-24-00_2016-06-01-00*.hdf')
str(DF_DIR)

In [ ]:
# data = dd.read_hdf(str(DF_DIR), key='dataset', chunksize=CHUNK_SIZE)
data = pd.read_hdf('/Volumes/CompanionEx/Data/dfs_pandas/PP_TS_2016-05-24-00_2016-06-01-00_0-200_20160624101922.hdf')

In [ ]:
datetime_index = data.index.get_level_values('datetime_start')
datetime_index = pd.DatetimeIndex(datetime_index)

data['day'] = datetime_index.to_period(freq='d')
data.head()

We can apply a query at this stage to limit the dataset.

## Split into train, test and validation sets

We will split by selecting a day as test and another as validation sets.

In [ ]:
from datetime import date, datetime, timedelta

In [ ]:
days = data['day'].value_counts()
days

In [ ]:
test_day = days.keys()[-2]
validation_day = days.keys()[-3]
print(test_day, validation_day)

In [ ]:
def filter_for_day(data, day, complement=False):
    datetime_index = data.index.get_level_values('datetime_start')
    datetime_index = pd.DatetimeIndex(datetime_index)

    if complement:
        return data[(datetime_index.year != day.year) | (datetime_index.month != day.month) | (datetime_index.day != day.day)]
    else:
        return data[(datetime_index.year == day.year) & (datetime_index.month == day.month) & (datetime_index.day == day.day)]


In [ ]:
test_data = filter_for_day(data, test_day)
validation_data = filter_for_day(data, validation_day)
train_data = filter_for_day(data, test_day, complement=True)  # Exclude the test day...
train_data = filter_for_day(train_data, validation_day, complement=True)  # ... and the validation day.

In [ ]:
print(data.shape, train_data.shape, test_data.shape, validation_data.shape)

In [ ]:
features = ['site_hash', 'timestamp_start', 'precipitation mm/h', 'temperature C', 'windspeed m/s']
targets = ['trafficspeed km/h']#, 'trafficflow counts/h']

Note that `site_hash` is the `mmh3.hash64` of the `site` column (the last component actually):

In [ ]:
mmh3.hash64('rws01_monibas_0010vwa0056ra')[-1]

In [ ]:
# features.npartitions  # Only for dask dataframes

As you can see we (lazy) loaded the entire dataset. It has been distributed into the above number of partitions.

## Modeling

In [ ]:
import lasagne  # Ignore any errors for now

In [ ]:
import numpy as np
import theano
import theano.tensor as T
# theano.config.exception_verbosity = 'high'

### Prepare batches

We model one sequence as whole-day measurement of a site. Batches are sets of such sequences of (possibly) various sizes. We expect a periodicity on the day level and try to fit a model to such behaviour.

We generate mini batches with the following function.

In [ ]:
from itertools import product
from random import shuffle

def batches(source_df, sites=None, days=None, max_batches=1000, max_batch_length=100):
    if sites is None:
        site_bag = set(source_df.index.get_level_values(0))
        
    if days is None:
        day_bag = set(source_df['day'].unique())
        
    sample_bag = product(sites, days)
#     sample_bag = list(sample_bag)  # Takes too long
#     shuffle(sample_bag)  # Takes too long

    for i in range(max_batches):
        samples = list()
        for j in range(max_batch_length):
            try:
                samples.append(next(sample_bag))
            except StopIteration:
                break
        
        if len(samples) == 0:
#             print("No samples at batch %i" % i)
            raise StopIteration
        
        # Prepare batch
        batch_length = len(samples)
        batch = np.zeros([batch_length, max_seq_length, len(features)], dtype='float64')
        mask = np.zeros([batch_length, max_seq_length], dtype='float64')
        target = np.zeros([batch_length, max_seq_length, len(targets)], dtype='float64')
        
        for j in range(batch_length):
            site, period = samples[j]
            
            # query measurements
            data = source_df.query("site == '%s'" % site)
            data = data[data['day'] == period]
            
            data_f = data[features].values
            data_t = data[targets].values

            seq_length = data.shape[0]
            assert seq_length <= max_seq_length, "Error: sequence longer than `max_seq_length` found"

            batch[j, :seq_length, :] = data_f
            target[j, :seq_length, :] = data_t
            mask[j, :seq_length] = np.ones([seq_length])
            
        yield i, batch, target, mask


Time the batch generation:

In [ ]:
site_bag = set(train_data.index.get_level_values(0))

In [ ]:
tdt = pd.DatetimeIndex(train_data.index.get_level_values('datetime_start'))
day_bag = set(tdt.to_period(freq='d'))

In [ ]:
max_batches = 1000
max_batch_length = 15  # Maximum number os day-long measurement sequence (of one site) per batch

max_seq_length = 24*60 # Maximum number of measurements per site per day

In [ ]:
# for batch_num, batch, target, mask in batches(train_data, sites=site_bag, days=day_bag):
#     print(batch_num, batch.size)

### Create the input and target variables

In [ ]:
input_var = T.tensor3('input', dtype=theano.config.floatX)

In [ ]:
target_values = T.tensor3('target', dtype=theano.config.floatX)
# target_values = T.matrix('target', dtype=theano.config.floatX)

### Define the model

In [ ]:
l_in = lasagne.layers.InputLayer(shape=(None, None, 5), input_var=input_var, name='input_layer')

In [ ]:
l_mask = lasagne.layers.InputLayer(shape=(None,None), name='mask')

In [ ]:
help(lasagne.layers.LSTMLayer)

In [ ]:
num_lstm_units = 20
max_grad = 5.0
l_lstm = lasagne.layers.LSTMLayer(l_in, num_units=num_lstm_units,
                                  gradient_steps=-1, grad_clipping=max_grad, unroll_scan=False,
                                  mask_input=l_mask, name='l_lstm_1')

In [ ]:
# We want to combine the LSTM with a dense layer and need to reshape the input. We dot this with a `ReshapeLayer`
help(lasagne.layers.ReshapeLayer)

In [ ]:
# First, retrieve symbolic variables for the input shape
n_batch, n_time_steps, n_features = l_in.input_var.shape

# Now, squash the n_batch and n_time_steps dimensions
l_reshape_in = lasagne.layers.ReshapeLayer(l_lstm, (-1, num_lstm_units)) 

In [ ]:
# Now, we can apply feed-forward layers as usual.
l_dense_1 = lasagne.layers.DenseLayer(l_reshape_in, num_units=20, nonlinearity=lasagne.nonlinearities.tanh, name='l_dense_1')
l_dense_2 = lasagne.layers.DenseLayer(l_dense_1, num_units=1, nonlinearity=lasagne.nonlinearities.tanh, name='l_dense_2')
# Now, the shape will be n_batch*n_timesteps, 1.  We can then reshape to
# n_batch, n_timesteps to get a single value for each timstep from each sequence
l_reshape_out = lasagne.layers.ReshapeLayer(l_dense_2, (n_batch, n_time_steps, 1), name='output_layer')

### Training

In [ ]:
# lasagne.layers.get_output produces an expression for the output of the net
network_output = lasagne.layers.get_output(l_reshape_out)
# The value we care about is the final value produced for each sequence
# so we simply slice it out.
predicted_values = network_output#[:, -1]

In [ ]:
# Our cost will be mean-squared error
# help(lasagne.objectives.squared_error)
loss = T.mean(lasagne.objectives.squared_error(predicted_values, target_values))
# loss = T.mean((predicted_values - target_values)**2)

In [ ]:
# Retrieve all parameters from the network
all_params = lasagne.layers.get_all_params(l_reshape_out)
# all_params

In [ ]:
# Compute adam updates for training
help(lasagne.updates.adam)
updates = lasagne.updates.adam(loss, all_params)

Theano functions for training computing cost and inference

In [ ]:
train = theano.function([l_in.input_var, target_values, l_mask.input_var], loss, updates=updates)

In [ ]:
compute_cost = theano.function([l_in.input_var, target_values, l_mask.input_var], loss)

In [ ]:
ide = theano.function([target_values], outputs=[target_values])

In [ ]:
ff = theano.function([l_in.input_var, l_mask.input_var], outputs=[predicted_values])

Check the shapes

In [ ]:
batch_num, batch, target, mask = next(iter(batches(train_data, sites=site_bag, days=day_bag)))

In [ ]:
print(batch.shape, target.shape, mask.shape)

In [ ]:
o = ff(batch, mask)[0]

In [ ]:
o.shape

In [ ]:
t = ide(target)[0]

In [ ]:
t.shape

In [ ]:
train(batch, target, mask)

Perform the training

In [ ]:
test_sites = set(test_data.index.get_level_values(0))

In [ ]:
max_batches = 1000
max_batch_length = 100  # Maximum number os day-long measurement sequence (of one site) per batch

max_seq_length = 24*60 # Maximum number of measurements per site per day

In [ ]:
# We'll train the network with 10 epochs of a maximum of `max_batches` each
num_epochs = 10
for epoch in range(num_epochs):
    print('TRAIN', end=' ')
    for batch_num, batch, target, mask in batches(train_data, sites=site_bag, days=day_bag,
                                                  max_batches=max_batches, max_batch_length=max_batch_length):
        train(batch, target, mask)
        if batch_num % 10 == 0:
            print(".", end='')
            if batch_num % 100 == 0:
                print(batch_num, end='')
    
    print('')

    cost_val = 0.0
    print('TEST', end=' ')
    for batch_num, batch, target, mask in batches(test_data, days=set((test_day,)), sites=test_sites):
        cost_val += compute_cost(batch, target, mask)
        if batch_num % 10 == 0:
            print(batch_num, end='')

    cost_val = cost_val/(batch_num + 1)
    print('')
    
    print("Epoch {} validation cost = {}".format(epoch + 1, cost_val))

### Validation